## 차량 대수별 성능평가

- 기본전제 : 배차주기 (60초) / 우회율 / 대기시간 / 차량정원 / 삽입창 개수 등 모두 동일
- 오로지 차량대수만 다름
- 우회율 = (DRT 실제 탑승 시간) / (OSRM 최단 주행 시간)

In [1]:
# ==== 시나리오 성능 비교 (이미지 공식: 시간 기반 우회율 적용 버전) ====
import os, json, time, math
from pathlib import Path
import pandas as pd
import requests
from collections import defaultdict

# --------- 사용자 설정 ---------
# 1~1366 순차 번호가 적용된 파케이 파일 경로
PARQUET_PATH = "../data/processed/240307_wirye_od.parquet"
# OSRM 서버 주소 (현재 실행 중인 8000번 포트 반영)
OSRM_BASE = "http://127.0.0.1:8000" 

# 분석할 시나리오 경로 설정
SCENARIOS = {
    "10대": "../outputs/10대",
    "20대": "../outputs/20대",
    "30대": "../outputs/30대",
    "40대": "../outputs/40대",
    "50대": "../outputs/50대", 
}

# --------- 유틸 함수 ---------
def jload(p):
    if not os.path.exists(p): return None
    with open(p, "r", encoding="utf-8") as f:
        try: return json.load(f)
        except:
            f.seek(0); return [json.loads(line) for line in f if line.strip()]

def get_list(x):
    if isinstance(x, list): return x
    if isinstance(x, dict):
        for k in ("events","moves"):
            if isinstance(x.get(k), list): return x[k]
    return []

_OSRM_TIME_CACHE = {}
def osrm_route_duration_sec(lon1, lat1, lon2, lat2):
    """OSRM에서 최단 도로 주행 시간(초)을 가져옴"""
    if None in (lon1, lat1, lon2, lat2): return 0.0
    key = (round(float(lon1),6), round(float(lat1),6), round(float(lon2),6), round(float(lat2),6))
    if key in _OSRM_TIME_CACHE: return _OSRM_TIME_CACHE[key]
    
    url = f"{OSRM_BASE}/route/v1/driving/{lon1},{lat1};{lon2},{lat2}"
    try:
        r = requests.get(url, params={"overview":"false"}, timeout=2.0)
        if r.status_code == 200:
            data = r.json()
            if data.get("routes"):
                sec = data["routes"][0]["duration"] # OSRM 기본 단위: 초(seconds)
                _OSRM_TIME_CACHE[key] = sec
                return sec
    except: pass
    return 0.0 

# --------- ① 원본 데이터 로드 ---------
# 요청 시각과 OD 좌표를 가져와서 대기시간 및 최단시간 계산에 사용
pq = pd.read_parquet(PARQUET_PATH).copy()
pq["KEY1"] = pq["KEY1"].astype(str)
req_df = pd.DataFrame({
    "req_id": pq["KEY1"],
    "t_request": pd.to_numeric(pq["승차_timestamp"], errors="coerce"),
    "o_lon": pd.to_numeric(pq["출발_x"], errors="coerce"), "o_lat": pd.to_numeric(pq["출발_y"], errors="coerce"),
    "d_lon": pd.to_numeric(pq["도착_x"], errors="coerce"), "d_lat": pd.to_numeric(pq["도착_y"], errors="coerce")
})
req_df = req_df.dropna(subset=["t_request"])

# --------- ② 시나리오별 분석 루프 ---------
rows = []
for scen, folder in SCENARIOS.items():
    if not os.path.exists(folder): continue
    events = get_list(jload(os.path.join(folder, "events.json")))
    summary = jload(os.path.join(folder, "summary.json")) or {}

    # 1. 배차 성공 여부 및 대기시간 계산
    pick_times = {str(e["req_id"]): float(e["t"]) for e in events if str(e.get("type")).upper()=="PICKUP"}
    num_passengers = len(pick_times)
    
    pick_df = pd.DataFrame({"req_id": list(pick_times.keys()), "t_pickup": list(pick_times.values())})
    merged = req_df.merge(pick_df, on="req_id")
    # 대기시간 = 실제 승차시각 - 승차 요청시각
    merged["wait_sec"] = (merged["t_pickup"] - merged["t_request"]).clip(lower=0)
    
    # 2. 우회율 계산 (이미지 공식: DRT 탑승시간 / 최단 도로 주행 시간)
    passengers = {} # 개별 승객의 PICKUP/DROPOFF 정보 저장
    for e in events:
        etype = str(e.get("type")).upper()
        rid = str(e.get("req_id"))
        if rid == "None": continue
        if etype == "PICKUP":
            passengers[rid] = {"t_pick": float(e.get("t")), "o": (e["lon"], e["lat"])}
        elif etype == "DROPOFF" and rid in passengers:
            passengers[rid]["t_drop"] = float(e.get("t"))
            passengers[rid]["d"] = (e["lon"], e["lat"])

    individual_detours = []
    for rid, p in passengers.items():
        if "t_drop" not in p: continue
        
        # 분자: 실제 탑승 시간(Ride Time) = 하차 시각 - 승차 시각
        actual_ride_time = p["t_drop"] - p["t_pick"]
        
        # 분모: 최단 도로 주행 시간(Shortest Duration)
        shortest_duration = osrm_route_duration_sec(p["o"][0], p["o"][1], p["d"][0], p["d"][1])
        
        if shortest_duration > 5: # 매우 짧은 거리는 수치 왜곡 방지를 위해 5초 이상만 계산
            ratio = actual_ride_time / shortest_duration
            # 우회율은 이론상 1.0 미만일 수 없으므로(도로망 최단시간 기준) 최소값 1.0 보정
            individual_detours.append(max(1.0, ratio))
            
    avg_detour_ratio = round(sum(individual_detours) / len(individual_detours), 3) if individual_detours else 0.0

    # 결과 취합
    rows.append({
        "시나리오": scen,
        "배차성공인원": num_passengers,
        "배차성공율(%)": round(num_passengers / len(req_df) * 100, 2),
        "미수행 요청": len(summary.get("rejected", [])),
        "평균 대기시간(초)": round(merged["wait_sec"].mean(), 1) if not merged.empty else 0,
        "최대 대기시간(초)": round(merged["wait_sec"].max(), 1) if not merged.empty else 0,
        "우회율": avg_detour_ratio
    })

# --------- ③ 최종 결과 출력 ---------
result_df = pd.DataFrame(rows).sort_values("시나리오").reset_index(drop=True)
result_df

,시나리오,배차성공인원,배차성공율(%),미수행 요청,평균 대기시간(초),최대 대기시간(초),우회율
0,10대,495,36.24,871,394.5,1233.0,2.276
1,20대,813,59.52,553,330.0,1087.0,2.319
2,30대,1046,76.57,320,300.0,1218.0,2.345
3,40대,1236,90.48,130,243.1,1187.0,2.187
4,50대,1319,96.56,47,201.0,1151.0,1.975
